In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input
from alibi_detect.cd import ClassifierDrift
import numpy as np
import methods


import logging 
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
# Assume that you have 12GB of GPU memory and want to allocate ~4GB:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.05)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

Num GPUs Available:  1


In [2]:
inlier_names = ["cCry", "sCry", "uCry", "hCast", "nhCast", "nsEC", "sEC", "WBC", "RBC", "Artifact", "Dirt", "LD"]
testImages_cCry = methods.getTestRawImages("cCry", (32,32))
testImages_sCry = methods.getTestRawImages("sCry", (32,32))
testImages_uCry = methods.getTestRawImages("uCry", (32,32))
testImages_hCast = methods.getTestRawImages("hCast", (32,32))
testImages_nhCast = methods.getTestRawImages("nhCast", (32,32))
testImages_nsEC = methods.getTestRawImages("nsEC", (32,32))
testImages_sEC = methods.getTestRawImages("sEC", (32,32))
testImages_WBC = methods.getTestRawImages("WBC", (32,32))
testImages_RBC = methods.getTestRawImages("RBC", (32,32))
testImages_Artifact = methods.getTestRawImages("Artifact", (32,32))
testImages_Dirt = methods.getTestRawImages("Dirt", (32,32))
testImages_LD = methods.getTestRawImages("LD", (32,32))

X_inliers = np.concatenate((testImages_cCry, testImages_sCry, testImages_uCry, testImages_hCast, testImages_nhCast, testImages_nsEC,
                       testImages_sEC, testImages_WBC, testImages_RBC))
unclassified_imgs = methods.getTestRawImages("Unclassified", (32,32))

In [3]:
model = tf.keras.Sequential(
  [
      Input(shape=(32, 32, 1)),
      Conv2D(8, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(16, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(32, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(2, activation='softmax')
  ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 8)         136       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 16)          2064      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 32)          8224      
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 11,450
Trainable params: 11,450
Non-trainable params: 0
_________________________________________________________________


In [4]:
cd = ClassifierDrift(X_inliers, model, p_val=.05, preds_type='probs', learning_rate=1e-3, epochs=5)

In [5]:
wbc_preds = cd.predict(testImages_WBC[:10], return_p_val = True, return_distance = True)
print(wbc_preds)

{'data': {'is_drift': 0, 'distance': 0.4039136979427998, 'p_val': 0.5962888665997994, 'threshold': 0.05}, 'meta': {'name': 'ClassifierDriftTF', 'detector_type': 'offline', 'data_type': None, 'params': {'binarize_preds ': False, 'preds_type': 'probs'}, 'backend': 'tensorflow'}}


In [6]:
rbc_preds = cd.predict(testImages_RBC[:10], return_p_val = True, return_distance = True)
print(rbc_preds)

{'data': {'is_drift': 0, 'distance': 0.04568273092369478, 'p_val': 0.9522793166252395, 'threshold': 0.05}, 'meta': {'name': 'ClassifierDriftTF', 'detector_type': 'offline', 'data_type': None, 'params': {'binarize_preds ': False, 'preds_type': 'probs'}, 'backend': 'tensorflow'}}


In [7]:
artifact_preds = cd.predict(testImages_Artifact[:10], return_p_val = True, return_distance = True)
print(artifact_preds)

{'data': {'is_drift': 0, 'distance': 0.464859437751004, 'p_val': 0.31983758601444073, 'threshold': 0.05}, 'meta': {'name': 'ClassifierDriftTF', 'detector_type': 'offline', 'data_type': None, 'params': {'binarize_preds ': False, 'preds_type': 'probs'}, 'backend': 'tensorflow'}}


In [8]:
dirt_preds = cd.predict(testImages_Dirt[:50], return_p_val = True, return_distance = True)
print(dirt_preds)

{'data': {'is_drift': 0, 'distance': 0.5301507537688442, 'p_val': 0.0716629076521975, 'threshold': 0.05}, 'meta': {'name': 'ClassifierDriftTF', 'detector_type': 'offline', 'data_type': None, 'params': {'binarize_preds ': False, 'preds_type': 'probs'}, 'backend': 'tensorflow'}}
